In [26]:
import pandas as pd
import sqlite3
import gc

In [27]:
# Function to drop columns if they exist in the DataFrame
def drop_columns_if_exist(df, columns_to_drop):
    columns_to_drop_existing = [col for col in columns_to_drop if col in df.columns]
    return df.drop(columns=columns_to_drop_existing, errors='ignore')

# Function to convert and clean DataFrame
def convert_and_clean_dataframe(df, dtype_dict):
    total_rows = len(df)
    rows_dropped = 0
    drop_indices = []

    for column, dtype in dtype_dict.items():
        if column in df.columns:
            try:
                if dtype == 'datetime64[ns]':
                    df[column] = pd.to_datetime(df[column], errors='coerce')
                elif dtype == 'float32':
                    df[column] = pd.to_numeric(df[column], errors='coerce').astype('float32')
                elif dtype == 'int8':
                    df[column] = pd.to_numeric(df[column], errors='coerce').astype('int8')
                elif dtype == 'int16':
                    df[column] = pd.to_numeric(df[column], errors='coerce').astype('int16')
                elif dtype == 'bool':
                    df[column] = df[column].replace({1: True, 0: False, 'true': True, 'false': False, 'True': True, 'False': False}).astype('boolean')
                elif dtype == 'category':
                    df[column] = df[column].astype('category')
            except Exception as e:
                print(f"Error converting column '{column}': {e}")
                drop_indices.extend(df.index[df[column].isna()])

    drop_indices = set(drop_indices)
    df = df.drop(index=drop_indices)
    rows_dropped = len(drop_indices)
    print(f"Total rows: {total_rows}")
    print(f"Number of rows dropped: {rows_dropped}")

    return df

# Drop all NaN columns
def drop_na_columns(df):
    return df.dropna(axis=1, how='all')

In [28]:
# Initialize garbage collection
gc.collect()

# auth id
# all authentication_id

# Read in the parquet files
df = pd.read_parquet('all_2023q3_2024q1_FULL.parquet')
df_auth = pd.read_parquet('authenticated_2023q3_2024q1_FULL.parquet')

# df = pd.read_csv('all_2023q3_2024q1_FULL.csv')
# df_auth = pd.read_csv('authenticated_2023q3_2024q1_FULL.csv')


# Get the column names of each DataFrame
columns_df = set(df.columns)
columns_df_auth = set(df_auth.columns)

# Find the common columns
common_columns = columns_df.intersection(columns_df_auth)


C:\Users\User\AppData\Local\Temp\ipykernel_23532\751461843.py:8: DtypeWarning: Columns (3,7,9,11,12,13,14,20,39,47,52,53,54,55,62,70,71,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('all_2023q3_2024q1_FULL.csv')
C:\Users\User\AppData\Local\Temp\ipykernel_23532\751461843.py:9: DtypeWarning: Columns (43,50,52,60,73,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_auth = pd.read_csv('authenticated_2023q3_2024q1_FULL.csv')


In [29]:
# Drop NaN columns
df = drop_na_columns(df)
df_auth = drop_na_columns(df_auth)

# Columns to drop
columns_to_drop = [
    'callback_url', 'credit_card_token', 'order', 'order_id', 'return_url', 'payment_method_id',
    'payment_processor_response', 'ucaf_authentication_data', 'ucaf_collection_indicator', 'xid',
    'language', 'merchant_url', 'three_ds_method_data', 'is_auto_timeout', 'token_type',
    'request_token', 'merchant_name', 'merchant_reference_code', 'descriptor', 'metadata',
    'internal_metadata', 'rewards', 'promotion', 'vat_amount', 'api_version', 'authorization_id',
    'authorization_additional_data', 'authorization_receipt_number', 'authorization_request_token',
    'assessment_id', 'authorization_response', 'capture_response', 'mid_label', 'network_token_id',
    'processor_response', 'retrieval_reference_number', 'card_data_id', 'capture_request_id',
    'masked_card_number', 'original_id', 'installment', 'fee_label', 'transaction_id', 'version',
    'authentication_id', 'authorization_request_id', 'system_trace_audit_number', 'is_cvn_submitted',
    'pares_status', 'processor_type', 'proof_xml', 'redirect_html', 'md', 'pa_req',
    'referer', 'request_id', 'term_url', 'three_ds_result','three_ds_version', 'user_agent', 'veres_enrolled', 
    'x_forwarded_for', 'initiate_three_ds_url', 'authentication_verification_response', 'authentication_transaction_id',
    'acs_url','initial_client_type', 'directory_server_transaction_id', 'billing_details', 'cc_version'
]

df = drop_columns_if_exist(df, columns_to_drop)
df_auth = drop_columns_if_exist(df_auth, columns_to_drop)

# Replace non-finite values (NaN, inf) with a placeholder (e.g., -1)
df['card_expiration_month'] = pd.to_numeric(df['card_expiration_month'], errors='coerce').fillna(-1).astype(int)
df['card_expiration_year'] = pd.to_numeric(df['card_expiration_year'], errors='coerce').fillna(-1).astype(int)


print(f"df shape: {df.shape}")
print(f"df_auth shape: {df_auth.shape}")


df shape: (391604, 51)
df_auth shape: (475066, 30)


In [30]:
# Define dtypes for conversion
df_dtypes = {
    'id': 'category', 'acquiring_bank_name': 'category', 'approval_code': 'category',
    'authorization_transaction_date': 'datetime64[ns]', 'authorized_amount': 'float32',
    'avs_code': 'category', 'bank_merchant_id': 'category', 'bank_reconciliation_id': 'category',
    'business_id': 'category', 'capture_amount': 'float32', 'card_brand': 'category',
    'card_expiration_month': 'int8', 'card_expiration_year': 'int16', 'card_holder_name': 'category',
    'card_type': 'category', 'cavv': 'category', 'cc_version': 'category', 'charge_type': 'category',
    'client_id': 'category', 'client_type': 'category', 'country': 'category', 'country_code': 'category',
    'created': 'datetime64[ns]', 'credit_card_payment_channel': 'category', 'credit_card_processor': 'category',
    'credit_card_token_id': 'category', 'currency': 'category', 'cvn_code': 'category', 'eci': 'category',
    'external_id': 'category', 'failure_reason': 'category', 'fee_amount': 'float32', 'is_blocked_by_fraud': 'bool',
    'is_switcher': 'bool', 'is_t4': 'bool', 'issuing_bank_name': 'category', 'merchant_id': 'category',
    'refund_status': 'category', 'requester_email': 'category', 'reversed_amount': 'float32',
    'settlement_status': 'category', 'settlement_updated': 'datetime64[ns]', 'should_authenticate_credit_card': 'bool',
    'should_settle_directly': 'bool', 'status': 'category', 'total_refund_amount': 'float32',
    'total_refund_fee_amount': 'float32', 'transaction_channel': 'category', 'ucaf': 'category',
    'updated': 'datetime64[ns]', 'user_id': 'category', 'use_reward': 'category', 'dt': 'datetime64[ns]',
    'amount': 'float32', 'authentication_type': 'category', 'card_bank': 'category', 'commerce_indicator': 'category',
    'credit_card_enrollment_info': 'category', 'cybersource_merchant_id': 'category', 'eci_raw': 'category', 'ip_address': 'category',
    'is_enrolled': 'bool', 'cof_type': 'category' 
    
}

# Apply conversions
df = convert_and_clean_dataframe(df, df_dtypes)
df_auth = convert_and_clean_dataframe(df_auth, df_dtypes)

C:\Users\User\AppData\Local\Temp\ipykernel_23532\3579349412.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column], errors='coerce')


Total rows: 391604
Number of rows dropped: 0
Total rows: 475066
Number of rows dropped: 0


In [31]:
df = df.dropna(axis=1, how='all')
df_auth = df_auth.dropna(axis=1, how='all')

In [32]:
# SQLite database file paths
db_df = 'processed_data.db'
db_df_auth = 'processed_authenticated_data.db'

# # Save DataFrames to SQLite databases
# with sqlite3.connect(db_df) as conn:
#     df.to_sql('data', conn, if_exists='replace', index=False)

# with sqlite3.connect(db_df_auth) as conn:
#     df_auth.to_sql('authenticated_data', conn, if_exists='replace', index=False)

# # Clean up memory
# gc.collect()

# print("DataFrames have been stored as SQLite databases.")

In [33]:
# Query data for January
query = """
SELECT * FROM data
WHERE created BETWEEN '2024-01-01' AND '2024-01-31'
"""

with sqlite3.connect(db_df) as conn:
    df_january = pd.read_sql_query(query, conn)

print(df_january.head())

# Similarly, for the authenticated_data database
query_auth = """
SELECT * FROM authenticated_data
WHERE created BETWEEN '2024-01-01' AND '2024-01-31'
"""

with sqlite3.connect(db_df_auth) as conn:
    df_auth_january = pd.read_sql_query(query_auth, conn)

print(df_auth_january.head())

                         id acquiring_bank_name approval_code  \
0  659fa7e925318000172adce8                 BRI          None   
1  659ff3b0a92e00001739b745             PAYMAYA          None   
2  659ffff56b277d0015703571                 BRI          None   
3  659f47fe6b277d00156f4807             MANDIRI        003216   
4  659fc31097c57900179939d9           GLOBALPAY        395526   

   authorized_amount avs_code bank_merchant_id bank_reconciliation_id  \
0          5028000.0     None     001988030000                   None   
1             8800.0        U        103496912                   None   
2          1065000.0     None     001988030000                   None   
3         11774700.0        U      71000832716              570803281   
4             1318.0     None     073006686104                   None   

                business_id  capture_amount  card_brand  ...  \
0  6450899466adc6a21e7bfde7       5028000.0        VISA  ...   
1  651b7f011256decfdac0b60d          8800.

In [36]:
merged_db = 'merged_data.db'

# Connect to the new database (which will also attach the other databases)
with sqlite3.connect(merged_db) as conn:
    # Attach the other databases
    conn.execute(f"ATTACH DATABASE '{db_df}' AS data_db")
    conn.execute(f"ATTACH DATABASE '{db_df_auth}' AS auth_db")
    
    # Create a new table with merged data
    conn.execute('''
    CREATE TABLE merged_data AS
    SELECT *
    FROM data_db.data
    LEFT JOIN auth_db.authenticated_data
    ON data_db.data.credit_card_token_id = auth_db.authenticated_data.credit_card_token_id
    UNION
    SELECT *
    FROM auth_db.authenticated_data
    LEFT JOIN data_db.data
    ON auth_db.authenticated_data.credit_card_token_id = data_db.data.credit_card_token_id;
    ''')

    # Commit the changes
    conn.commit()

print("Tables have been merged and stored in the new SQLite database.")

Tables have been merged and stored in the new SQLite database.


In [38]:
# Connect to the merged database and query the data
with sqlite3.connect(merged_db) as conn:
    query = "SELECT * FROM merged_data;"
    df = pd.read_sql_query(query, conn)

(926437, 80)